# Lesson 23 - Spark SQL

### Introduction

In additional to the DataFrame API that Spark provides for working with structured data, it also provides tools for submitting SQL queries against DataFrames we have created, or against tables we have stored in a database. Spark allows us to switch between these modes of operation seamlessly.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder.getOrCreate()

### Creating Tables

We can register a Spark DataFrame as a temporary table using the `createOrReplaceTempView()` method.

In [0]:
diamonds = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .schema(
        'carat DOUBLE, cut STRING, color STRING, clarity STRING, depth DOUBLE, '
        'table DOUBLE, price INTEGER, x DOUBLE, y DOUBLE, z DOUBLE'
    )
    .csv('/FileStore/tables/diamonds.txt')
)

diamonds.printSchema()

root
-- carat: double (nullable = true)
-- cut: string (nullable = true)
-- color: string (nullable = true)
-- clarity: string (nullable = true)
-- depth: double (nullable = true)
-- table: double (nullable = true)
-- price: integer (nullable = true)
-- x: double (nullable = true)
-- y: double (nullable = true)
-- z: double (nullable = true)

In the cell below, we check to see what tables are available in the SQL database attached to our cluster.

In [0]:
print(spark.catalog.listTables())

[]

We will now register the `diamonds` DataFrame as a temporary table in the database, and will then submit SQL queries against this table.

In [0]:
diamonds.createOrReplaceTempView('diamonds')
print(spark.catalog.listTables())

[Table(name='diamonds', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

### Submitting SQL Commands

We can submit SQL queries using the `sql()` method of the `SparkSession` object. The cell below deletes the table we just created.

In [0]:
spark.sql('DROP TABLE diamonds')
print(spark.catalog.listTables())

[]

We will now add the table back to the database before continuing on.

In [0]:
diamonds.createOrReplaceTempView('diamonds')
print(spark.catalog.listTables())

[Table(name='diamonds', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

### SELECT Statements

We can use the SELECT keyword in SQL to select specific columns from a table. The SELECT keyword in SQL behaves similarly to the `select()` DataFrame method. 
Note that the `sql()` method returns a Spark DataFrame, so we will need to use `show()` to display the results.

In [0]:
spark.sql('SELECT * FROM diamonds').show(5)

+-----+-------+-----+-------+-----+-----+-----+----+----+----+
carat| cut|color|clarity|depth|table|price| x| y| z|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
 0.23| Ideal| E| SI2| 61.5| 55.0| 326|3.95|3.98|2.43|
 0.21|Premium| E| SI1| 59.8| 61.0| 326|3.89|3.84|2.31|
 0.23| Good| E| VS1| 56.9| 65.0| 327|4.05|4.07|2.31|
 0.29|Premium| I| VS2| 62.4| 58.0| 334| 4.2|4.23|2.63|
 0.31| Good| J| SI2| 63.3| 58.0| 335|4.34|4.35|2.75|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
only showing top 5 rows

In [0]:
spark.sql('SELECT carat, cut, color, clarity, price  FROM diamonds').show(5)

+-----+-------+-----+-------+-----+
carat| cut|color|clarity|price|
+-----+-------+-----+-------+-----+
 0.23| Ideal| E| SI2| 326|
 0.21|Premium| E| SI1| 326|
 0.23| Good| E| VS1| 327|
 0.29|Premium| I| VS2| 334|
 0.31| Good| J| SI2| 335|
+-----+-------+-----+-------+-----+
only showing top 5 rows

SQL provides a few aggregating functions that we can use in SELECT statements. Some examples are `COUNT()`, `SUM()`, `MIN()`, `MAX()`, and `AVG()`.

In [0]:
spark.sql('SELECT COUNT(*) AS num_diamonds FROM diamonds').show()

+------------+
num_diamonds|
+------------+
 53940|
+------------+

SQL also has a `ROUND()` function that we can use in our queries.

In [0]:
spark.sql(
    'SELECT COUNT(*) as n,\
            ROUND(AVG(price),2) AS avg_price,\
            ROUND(AVG(carat),2) AS avg_carat\
     FROM diamonds'
).show()

+-----+---------+---------+
 n|avg_price|avg_carat|
+-----+---------+---------+
53940| 3932.8| 0.8|
+-----+---------+---------+

The DISTINCT keyword returns a list of unique values in the specified column.

In [0]:
spark.sql('SELECT DISTINCT cut FROM diamonds').show()

+---------+
 cut|
+---------+
 Premium|
 Ideal|
 Good|
 Fair|
Very Good|
+---------+

In [0]:
spark.sql('SELECT DISTINCT color FROM diamonds').show()

+-----+
color|
+-----+
 F|
 E|
 D|
 J|
 G|
 I|
 H|
+-----+

The cell below shows how to use SQL to count the number of unique values appearing in a particular column. The cell also shows two other ways of performing the same task using DataFrame methods.

In [0]:
spark.sql('SELECT COUNT(DISTINCT cut) AS num_cuts\
           FROM diamonds').show()

#diamonds.select(expr('COUNT(DISTINCT cut) AS num_cuts')).show()

+--------+
num_cuts|
+--------+
 5|
+--------+

### ORDER BY Clause

We can use the ORDER BY keywork in SQL to sort the rows resulting from a SELECT statement.

In [0]:
spark.sql(
    'SELECT DISTINCT cut\
     FROM diamonds\
     ORDER BY cut DESC'
).show()

+---------+
 cut|
+---------+
Very Good|
 Premium|
 Ideal|
 Good|
 Fair|
+---------+

### WHERE Clause

The WHERE keyword in SQL can be used to filter rows out of a table. This is similar to the `filter()` DataFrame method. In fact, DataFrames also have a `where()` method that performs exactly the same function as `filter()`.

In [0]:
spark.sql(
    'SELECT COUNT(*) as num_ideal\
     FROM diamonds\
     WHERE cut == "Ideal"'
).show()

#diamonds.filter(expr('cut == "Ideal"')).select(expr('COUNT(*) AS num_ideal')).show()

+---------+
num_ideal|
+---------+
 21551|
+---------+

In [0]:
spark.sql(
    'SELECT count(*) AS num_ideal_e\
     FROM diamonds\
     WHERE cut == "Ideal" AND color == "E"'
).show()



+-----------+
num_ideal_e|
+-----------+
 3903|
+-----------+

### GROUP BY Clause

We can use the GROUP BY keyword in SQL to perform grouping and aggregations. The aggregating functions are specified in the SELECT statement.

In [0]:
spark.sql(
    'SELECT cut, MEAN(price) as avg_price\
     FROM diamonds\
     GROUP BY cut'
).show()

+---------+------------------+
 cut| avg_price|
+---------+------------------+
 Premium|4584.2577042999055|
 Ideal| 3457.541970210199|
 Good| 3928.864451691806|
 Fair| 4358.757763975155|
Very Good|3981.7598907465654|
+---------+------------------+

In [0]:
spark.sql(
    'SELECT cut,\
            COUNT(*) AS N,\
            ROUND(AVG(carat),2) AS avg_carat,\
            ROUND(AVG(price),0) AS avg_price\
     FROM diamonds\
     GROUP BY cut\
     ORDER BY avg_price DESC'
).show()

+---------+-----+---------+---------+
 cut| N|avg_carat|avg_price|
+---------+-----+---------+---------+
 Premium|13791| 0.89| 4584.0|
 Fair| 1610| 1.05| 4359.0|
Very Good|12082| 0.81| 3982.0|
 Good| 4906| 0.85| 3929.0|
 Ideal|21551| 0.7| 3458.0|
+---------+-----+---------+---------+

### User Defined Functions

We can register user defined functions to use in our SQL queries.

In [0]:
def carat_bin(x):
    bins = ['[0,1)', '[1,2)', '[2,3)', '[3,4)', '[4,5)', '[5,6)']
    return bins[int(x)]
 
print(carat_bin(2.1))

[2,3)

In [0]:
spark.udf.register('carat_bin', carat_bin)

spark.sql('SELECT carat, carat_bin(carat) AS carat_bin FROM diamonds ORDER BY carat DESC').show(10)

+-----+---------+
carat|carat_bin|
+-----+---------+
 5.01| [5,6)|
 4.5| [4,5)|
 4.13| [4,5)|
 4.01| [4,5)|
 4.01| [4,5)|
 4.0| [4,5)|
 3.67| [3,4)|
 3.65| [3,4)|
 3.51| [3,4)|
 3.5| [3,4)|
+-----+---------+
only showing top 10 rows

In [0]:
spark.sql(
    'SELECT carat_bin, count(*) AS N\
     FROM (SELECT carat_bin(carat) AS carat_bin FROM diamonds)\
     GROUP BY carat_bin\
     ORDER BY carat_bin'
).show()

+---------+-----+
carat_bin| N|
+---------+-----+
 [0,1)|34880|
 [1,2)|16906|
 [2,3)| 2114|
 [3,4)| 34|
 [4,5)| 5|
 [5,6)| 1|
+---------+-----+